In [ ]:
//Im Terminal einmal npm install mongodb ausführen damit das Notebook klappt bei Rückfragen einfach einmal melden - Leonard Kipar
let ObjectId = require("mongodb").ObjectId;
let MongoClient = require("mongodb").MongoClient;

In [ ]:
let mongourl = "mongodb://127.0.0.1:27017";

In [ ]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let movie = await movies.findOne({"title":"The Big Short"});
    
    console.log(movie);
    
    db.close();
});

In [ ]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let languages = await movies.aggregate([
        { 
            $match: {  
                language : { $regex: /^English/ }, 
            },
        },
        {
            $group: {
                _id: "$language",
                totalQty: { $count: {} },
            },
        },
        { 
            $sort: { totalQty: -1 } 
        },
    ]).toArray();
    console.log(languages);
    db.close();
});

In [ ]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let languages = await movies.aggregate([
        {
            $sortByCount: "$language",
        }
    ]).toArray();
    console.log(languages);
    db.close();
});

In [ ]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let cnt1Pipeline = [ { '$count': 'year' } ];
    let cnt2Pipeline = [ { '$count': 'director' } ];
    let cnt3Pipeline = [ { '$count': 'writer' } ];
    let languages = await movies.aggregate(cnt1Pipeline).toArray();
    console.log(languages);
    db.close();
});

In [ ]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let cnt1Pipeline = [{ 
        $group: {
            _id: "Jahre",
            minYear: { $min: "$year" },
            maxYear: { $max: "$year" },
        }
    }];
    let languages = await movies.aggregate(cnt1Pipeline).toArray();
    console.log(languages);
    db.close();
});

In [ ]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let years = await movies.aggregate([
        {
            $project: {
                _id: 1,
                year: 1
            },
        }
    ]).toArray();
    console.log(years.length);
    for(entries of years) {
        if(entries.year.length > 4) {
            await movies.updateOne({_id: ObjectId(entries._id.toString())},{$set: {year: parseInt(entries.year.split("�")[0])}})
        } else {
            await movies.updateOne({_id: ObjectId(entries._id.toString())},{$set: {year: parseInt(entries.year)}})
        }
    }
    db.close();
});

In [ ]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let cnt1Pipeline = [
        {
            $project: {
                //_id: 1,
                year: 1
            },
        }
    ];
    let years = await movies.aggregate(cnt1Pipeline).toArray();
    for(entries of years) {
        //console.log(entries.year.toString().length)
        if(entries.year.toString().length > 4) {
            console.log(entries.year);
        }
    }
    db.close();
});

In [ ]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let year2016 = [
        {
            $match: { year: 2016 }
        },
        {
            $sortByCount: "$year",
        }
    ];
    let years = await movies.aggregate(year2016).toArray();
    console.log(years);
    db.close();
});

In [ ]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let list = await movies.distinct("year");

    
    //Aggregate
    let start_timer = performance.now();
    for(yr of list) {
        let years = await movies.aggregate([
            {
                $match: {
                    year: yr
                }
            }
        ]);
    }
    let stop_timer = performance.now();
    console.log(`Aggregate Time: ${stop_timer-start_timer}ms`);
    //
    
    //Find
    let start_timer2 = performance.now();
    for(yr of list) {
        let find = await movies.find({year: yr});
    }
    let stop_timer2 = performance.now();
    console.log(`Find Time: ${stop_timer2-start_timer2}ms`);
    //
    
    db.close();
});

In [ ]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let movies2016 = await movies.aggregate([
        {
            $match: {
                year: 2016
            },
        },
        {  
            $group: {
                _id: "$rating",
                count: { $count: {} }
            },
        },
        {
            $sort: { count: -1 }
        }
    ]).toArray();
    console.log(movies2016);
    db.close();
});

In [ ]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let shortSince2000 = await movies.aggregate([
        {
            $match: {
                genre: { $regex: /Short/ },
                year: { $gte: 2000 }
            },
        },
        {  
            $group: {
                _id: "Short",
                count: { $count: {} }
            },
        }
    ]).toArray();
    console.log(shortSince2000);
    db.close();
});

In [ ]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    let movies = db.db("mflix").collection("movies");
    let shortSince2000 = await movies.aggregate([
        {
            $match: {
                genre: {$regex: /Short/ },
                year: { $gte: 2000 }
            },
        },
        {  
            $project: {
                title: "$title",
                year: "$year",
                genre: "$genre"
            },
        },
        {
            $sort: {
                title: 1
            }
        },
        {
            $out: { db: "ShortMovies", coll: "2000er"}
        }
    ]).toArray();
    //console.log(shortSince2000);
    //await db.db("ShortMovies").collection("2000er").insertMany(shortSince2000);
    db.close();
});

In [6]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    //Aufgabe 1
    let telephone = [];
    var dbo = db.db("Kontakte");
    for(let i = 0; i < 100000; i++) {
        let telephonenumber = {
            telephone: i
        }
        telephone.push(telephonenumber);
    }
    await dbo.collection("Telefonnummern").insertMany(telephone);
    db.close();
});

In [7]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    //Aufgabe 2 - Count
    var count = await db.db("Kontakte").collection("Telefonnummern").count({"telephone" : {"$gt": 50000}});
    console.log(count);
    db.close();
});

49999


In [ ]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    //Aufgabe 3 - Distinct
    var distinct1 = await db.db("tif20").collection("bike").distinct("rideable_type");
    var distinct2 = await (await db.db("tif20").collection("bike").distinct("end_station_name", {"start_station_name":"Willis Ave & E 143 St"}));
    console.log(distinct2);
    db.close();
});

In [11]:
MongoClient.connect(mongourl, async function(err, db) {
    if(err) throw err;
    //Aufgabe 4 - Aggregation
    /*var cofeeshop = await db.db("coffeeshop").collection("sales").insertMany([
        { "_id" : 1, "item" : "Americanos", "price" : 5, "size": "Short", "quantity" : 22, "date" : Date("2022-01-15T08:00:00Z") },
        { "_id" : 2, "item" : "Cappuccino", "price" : 6, "size": "Short","quantity" : 12, "date" : Date("2022-01-16T09:00:00Z") },
        { "_id" : 3, "item" : "Lattes", "price" : 15, "size": "Grande","quantity" : 25, "date" : Date("2022-01-16T09:05:00Z") },
        { "_id" : 4, "item" : "Mochas", "price" : 25,"size": "Tall", "quantity" : 11, "date" : Date("2022-02-17T08:00:00Z") },
        { "_id" : 5, "item" : "Americanos", "price" : 10, "size": "Grande","quantity" : 12, "date" : Date("2022-02-18T21:06:00Z") },
        { "_id" : 6, "item" : "Cappuccino", "price" : 7, "size": "Tall","quantity" : 20, "date" : Date("2022-02-20T10:07:00Z") },
        { "_id" : 7, "item" : "Lattes", "price" : 25,"size": "Tall", "quantity" : 30, "date" : Date("2022-02-21T10:08:00Z") },
        { "_id" : 8, "item" : "Americanos", "price" : 10, "size": "Grande","quantity" : 21, "date" : Date("2022-02-22T14:09:00Z") },
        { "_id" : 9, "item" : "Cappuccino", "price" : 10, "size": "Grande","quantity" : 17, "date" : Date("2022-02-23T14:09:00Z") },
        { "_id" : 10, "item" : "Americanos", "price" : 8, "size": "Tall","quantity" : 15, "date" : Date("2022-02-25T14:09:00Z")}
    ]);*/
    //Aggregation Example
    var coffeeshopAgg = await db.db("coffeeshop").collection("sales").aggregate([
        { 
            $match: { item: "Americanos" } 
        },
        { 
            $group: {
                _id: "$size",
                totalQty: {$sum: "$quantity"}
            }
        },
        {
            $sort: { totalQty : -1}		
        }
    ]).toArray();
    console.log(coffeeshopAgg)
    //Sum Example
    var coffeeshopSum = await db.db("coffeeshop").collection("sales").aggregate([
        {
            $group: {
                _id: "$item",
                totalQty: { $sum: '$quantity' },
            },
        },
        { 
            $sort: { totalQty: -1 } 
        },
    ]).toArray();
    console.log(coffeeshopSum)
    //Count Example
    var coffeeshopCount = await db.db("coffeeshop").collection("sales").aggregate([
        {
            $group: {
                _id: "$item",
                countCoffee: { $count: {} },
            },
        },
        { 
            $sort: { countCoffee: -1 } 
        },
    ]).toArray();
    console.log(coffeeshopCount);
    
    db.close();
});

[
  { _id: 'Grande', totalQty: 33 },
  { _id: 'Short', totalQty: 22 },
  { _id: 'Tall', totalQty: 15 }
]
[
  { _id: 'Americanos', totalQty: 70 },
  { _id: 'Lattes', totalQty: 55 },
  { _id: 'Cappuccino', totalQty: 49 },
  { _id: 'Mochas', totalQty: 11 }
]
[
  { _id: 'Americanos', countCoffee: 4 },
  { _id: 'Cappuccino', countCoffee: 3 },
  { _id: 'Lattes', countCoffee: 2 },
  { _id: 'Mochas', countCoffee: 1 }
]
